## comboのimport error 対策

In [1]:
!pip show combo

Name: combo
Version: 0.2
Summary: UNKNOWN
Home-page: UNKNOWN
Author: Tsuyoshi Ueno
Author-email: tsuyoshi.ueno@gmail.com
License: UNKNOWN
Location: /Users/mkumada/.pyenv/versions/3.6.8/envs/combo3/lib/python3.6/site-packages
Requires: 
Required-by: 


In [13]:
import sys
import pprint
pprint.pprint(sys.path)
sys.path.append('/Users/mkumada/.pyenv/versions/3.6.8/envs/combo3/lib/python3.6/site-packages/')
pprint.pprint(sys.path)

['/Users/mkumada/.pyenv/versions/3.6.8/lib/python36.zip',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6/lib-dynload',
 '',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6/site-packages',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6/site-packages/IPython/extensions',
 '/Users/mkumada/.ipython',
 '/Users/mkumada/.pyenv/versions/3.6.8/envs/combo3/lib/python3.6/site-packages/',
 '/Users/mkumada/.pyenv/versions/3.6.8/envs/combo3/lib/python3.6/site-packages/',
 './combo_single/']
['/Users/mkumada/.pyenv/versions/3.6.8/lib/python36.zip',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6/lib-dynload',
 '',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6/site-packages',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6/site-packages/IPython/extensions',
 '/Users/mkumada/.ipython',
 '/Users/mkumada/.pyenv/versions/3.6.8/envs/combo3/lib/python3.6/site-packag

## library import

In [5]:
import pprint
sys.path.append('/Users/mkumada/.pyenv/versions/3.6.8/envs/combo3/lib/python3.6/site-packages/')
pprint.pprint(sys.path)

['/Users/mkumada/Documents/class/2020/work/combo3/mywork',
 '/Users/mkumada/.vscode/extensions/ms-python.python-2020.8.108011/pythonFiles',
 '/Users/mkumada/.vscode/extensions/ms-python.python-2020.8.108011/pythonFiles/lib/python',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python36.zip',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6/lib-dynload',
 '',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6/site-packages',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6/site-packages/IPython/extensions',
 '/Users/mkumada/.ipython',
 '/Users/mkumada/.pyenv/versions/3.6.8/envs/combo3/lib/python3.6/site-packages/']


In [6]:
import numpy as np
import pickle
import scipy
import combo
import os
import urllib
import ssl
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
# 濃度変換
def concentration_conversion(X):
    X[:, 1:-1] = X[:, 1:-1]/np.expand_dims(np.sum(X[:, 1:-1], axis=1), axis=1)
    return X

In [8]:
# データ取得
def load_data(path):
    A = np.loadtxt(path,skiprows=1,delimiter=',')
    A = concentration_conversion(A)
    X = A[:, 1:-1]
    y  = -A[:, -1]
    return X, y

In [9]:
# delta
def cost_func(a, b, c):
    return (0.2-(a/2+b))**2 + (0.2-np.sqrt(3)/2)**2 + (c-0.15)**2

In [10]:
train_path = './data/train.csv'
X, y = load_data(train_path)
print('X:', X)
print('y:',y)

X: [[1.         0.         0.        ]
 [0.5        0.5        0.        ]
 [0.33333333 0.66666667 0.        ]
 [0.25       0.5        0.25      ]
 [0.4        0.4        0.2       ]]
y: [-0.5 -0.2 -1.  -1.4 -1.2]


In [11]:
# If you want to apply combo to other problems, you have to customize this class. 
class simulator:
    def __init__( self ):
        _, self.t = load_data(train_path)
    
    def __call__( self, action ):
        return self.t[action]

In [15]:
# Design of policy

# Declaring the policy by 
policy = combo.search.discrete.policy(test_X=X)
# test_X is the set of candidates which is represented by numpy.array.
# Each row vector represents the feature vector of the corresponding candidate

# set the seed parameter 
policy.set_seed(0)

In [16]:
# If you want to perform the initial random search before starting the Bayesian optimization, 
# the random sampling is performed by 

res = policy.random_search(max_num_probes=2, simulator=simulator())
# Input: 
# max_num_probes: number of random search 
# simulator = simulator
# output: combo.search.discreate.results (class)


# single query Bayesian search
# The single query version of COMBO is performed by 
res = policy.bayes_search(max_num_probes=2, simulator=simulator(), score='TS', 
                                                  interval=2, num_rand_basis=5000)

# Input
# max_num_probes: number of searching by Bayesian optimization
# simulator: the class of simulator which is defined above
# score: the type of aquision funciton. TS, EI and PI are available
# interval: the timing for learning the hyper parameter. 
#               In this case, the hyper parameter is learned at each 20 steps
#               If you set the negative value to interval, the hyper parameter learning is not performed 
#               If you set zero to interval, the hyper parameter learning is performed only at the first step
# num_rand_basis: the number of basis function. If you choose 0,  ordinary Gaussian process runs

NameError: name 'xrange' is not defined

In [17]:
# The result of searching is summarized in the class combo.search.discrete.results.history()
# res.fx: observed negative energy at each step
# res.chosed_actions: history of choosed actions
# fbest, best_action= res.export_all_sequence_best_fx(): current best fx and current best action 
# that has been observed until each step
# res.total_num_search: total number of search
print('f(x)=')
print(res.fx[0:res.total_num_search])
best_fx, best_action = res.export_all_sequence_best_fx()
print('current best')
print (best_fx)
print ('current best action=')
print (best_action)
print ('history of chosed actions=')
print (res.chosed_actions[0:res.total_num_search])

f(x)=


NameError: name 'res' is not defined

In [26]:
# save the results
res.save('./save/test.npz')

In [27]:
res.load('./save/test.npz')